In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Carga de archivos

df_clientes = pd.read_csv(r"data/clean/clientes.csv")
df_productos = pd.read_csv(r"data/clean/productos.csv")
df_ventas = pd.read_csv(r"data/clean/ventas.csv")
df_detalle = pd.read_csv(r"data/clean/detalle_ventas.csv")
df_features = pd.read_csv(r"data/clean/features_RFM.csv")

In [ ]:
df_features = df_features.rename(columns={"cluster": "segmento"})

# Segmentación RFM

Hemos completado la clasificación de nuestra base de clientes utilizando el modelo RFM (Recencia, Frecuencia, Valor Monetario), segmentando a nuestros compradores en cuatro grupos estratégicos: 
- **Fieles**: Representan nuestro segmento más valioso, garantizando la mayor parte de nuestros ingresos y deben ser premiados y retenidos.
- **En Ascenso**: Estos son nuestra principal oportunidad de crecimiento, son clientes activos a los que debemos motivar para que aumenten su frecuencia de compra y se conviertan en Fieles.
- **En Alerta**: Son clientes que solían comprar, pero su Recencia ha caído significativamente.
- **Congelados**: Representan clientes inactivos que no han comprado en un largo periodo.

Esta nueva segmentación nos proporciona un mapa claro sobre dónde debemos invertir nuestros recursos de marketing.

In [ ]:
df_features.groupby('segmento').agg(clientes=("id_cliente", "count"),
                                   recency_dias_promedio=("recency_dias", "mean"),
                                   frequency_promedio=("frequency", "mean"),
                                   monetary_promedio=("monetary", "mean")).round(2).reset_index()

#### **Segmento 0: "En Alerta"**

Recencia: 21 días. A pesar de haber comprado bastante (29 veces), 21 días es el límite donde el cliente empieza a "olvidar" el mini-market.

Frecuencia y Valor: Media/Baja. Aunque han gastado y comprado más que los Congelados, su valor es bajo comparado con los otros.

Conclusión: Este es el segmento de mayor riesgo e impacto. Representan una fuga de ingresos potencial masiva si no se detiene a tiempo.

#### **Segmento 1: "En Ascenso"**

Recencia: 2 días. Son extremadamente recientes.

Frecuencia y Valor: Altos (194 veces, 1.3M). Su frecuencia y gasto ya son significativos, lo que demuestra un alto nivel de lealtad.

Conclusión: Son nuestros Potenciales Campeones. La inversión en ellos debe enfocarse en programas de lealtad de nivel superior que los premien para subir al Cluster 3.

#### **Segmento 2: "Congelados"**

Recencia: 470 días (más de un año).

Frecuencia y Valor: 1 compra, $6,377.

Conclusión: Mantenerlos en la lista de bajo coste. Solo enviarles campañas masivas y muy económicas (ej. correos genéricos de ofertas estacionales) y enfocarnos en los grupos de mayor rentabilidad.

#### **Segmento 3: "Fieles"**

Recencia: 1 día.

Frecuencia y Valor: 499 veces, $3.5M.

Conclusión: El foco es Retención Cero Fugas. Debemos tratarlos como VIP, buscar su opinión constantemente (para mejorar la tienda) y utilizarlos como fuente de referidos y de validación de nuevos productos.

#### **Conclusión**
Como demuestran los resultados, tenemos una base de clientes con altos niveles de compromiso: el 96% del Valor Monetario total proviene de los segmentos 'En Ascenso' y 'Fieles', a pesar de que representan solo el 21% de nuestra base de datos (329 clientes). Esta élite es la que sostiene la rentabilidad de Los Latinos y debe ser nuestra máxima prioridad. Sin embargo, nuestro mayor riesgo reside en el Cluster 'En Alerta'. Con 1,075 clientes, este grupo es el más numeroso y su Recencia promedio de 21 días indica que están al borde de la inactividad. Si no implementamos una estrategia de reactivación urgente, corremos el riesgo de que esta gran masa de clientes se convierta en 'Congelados', erosionando gravemente nuestra base activa.

# Análisis de Segmentos

## Segmento 0: "En Alerta"

In [ ]:
df_ventas_seg = df_ventas.merge(df_features[['id_cliente', 'segmento']], on='id_cliente', how='left')

In [ ]:
ventas_seg_0 = df_ventas_seg[df_ventas_seg['segmento'] == 0].copy()

In [ ]:
ventas_seg_0['añomes'] = pd.to_datetime(ventas_seg_0['fecha']).dt.to_period('M')
ventas_seg_0['diasem'] = pd.to_datetime(ventas_seg_0['fecha']).dt.dayofweek

In [ ]:
ventas_seg_0

In [ ]:
ventas_seg_0.head(5)[['id_venta']].to_json(orient='records', indent=2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

# Cargar datos de ejemplo
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['species'] = iris.target

# Pair plot básico
sns.pairplot(df, hue='species', diag_kind='kde', palette='husl')
plt.show()

In [ ]:
ventas_seg_0.groupby("diasem").agg(prom_ventas_dia=('id_venta', 'count')).reset_index()

In [ ]:
ventas_seg_0.groupby('añomes').agg(ventas_diarias=('id_venta', 'count')).reset_index().plot(x='añomes', y='ventas_diarias', kind='line', title='Ventas diarias - Segmento 0')